<a href="https://colab.research.google.com/github/hugopoggi/enem/blob/main/impacto_pandemia_enem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=a069f85e83b4b3ee0f6e6ee11eadf79f97ce3cd470e26d8c8a0782ad516d05fc
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession as ss
from pyspark.sql import functions as f
from pyspark.sql.types import DoubleType

In [3]:
spark = ss.builder.appName("ENEM").getOrCreate()



---


*   Está análise tem como objetivo análisar o impacto da pandemia nas provas  do enem a estudantes de menos de 25 anos que concluiram o ensino médio no ano da prova ou seguinte.
*   Foram utilizados as bases de dados abertas do INEP das provas do enem de 2019 e 2020


---





**INICIANDO TRATAMENTO DE DADOS ENEM 2019
**

In [4]:
# Importando a base de dados do enem de 2019
url = "/content/drive/MyDrive/BasesSpark/enem/enem_2019/DADOS/MICRODADOS*"
enem_2019 =spark.read.csv(
    url,
    header=True,
    inferSchema=True,
    encoding= "UTF-8",
    sep=";"
)

In [5]:
enem_2019.show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+----------------+---------+---------+----------------------+------------------+---------------+------------------+------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_AN

In [6]:
enem_2019.count()

5095171

In [7]:
enem_2019.printSchema()

root
 |-- NU_INSCRICAO: long (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- TP_FAIXA_ETARIA: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: integer (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- TP_ST_CONCLUSAO: integer (nullable = true)
 |-- TP_ANO_CONCLUIU: integer (nullable = true)
 |-- TP_ESCOLA: integer (nullable = true)
 |-- TP_ENSINO: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_MUNICIPIO_ESC: integer (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: integer (nullable = true)
 |-- TP_LOCALIZACAO_ESC: integer (nullable = true)
 |-- TP_SIT_FUNC_ESC: integer (nullable = true)
 |-- CO_MUNICIPIO_PROVA: integer (nullable = true)
 |-- NO_MUNICIPIO_PROVA: string (nullable = true)
 |-- CO_UF_PROVA: integer 

In [8]:
# Filtrando alunos que estiveram presentes em todos os dias de prova, que nao concluiram ou irão concluir o ensino medio
enem_2019.where(
    "TP_FAIXA_ETARIA <= 10 and TP_PRESENCA_CN like '1' and TP_PRESENCA_CH like '1' and TP_PRESENCA_LC like '1' and TP_PRESENCA_MT like '1' and TP_ST_CONCLUSAO like '2' or TP_ST_CONCLUSAO like '3'"
).show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+--------------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO

In [9]:
# Aplicando alteração ao Data Frame
enem_2019 = enem_2019.where(
    "TP_FAIXA_ETARIA <= 10 and TP_PRESENCA_CN like '1' and TP_PRESENCA_CH like '1' and TP_PRESENCA_LC like '1' and TP_PRESENCA_MT like '1' and TP_ST_CONCLUSAO like '2' or TP_ST_CONCLUSAO like '3'"
)

In [10]:
enem_2019.show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+--------------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO

In [11]:
# Removendo notas nulas:
enem_2019.dropna(how='any',subset=["NU_NOTA_CN",
                                   "NU_NOTA_CH",
                                   "NU_NOTA_LC",
                                   "NU_NOTA_MT",
                                   "NU_NOTA_REDACAO"]).show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+--------------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO

In [12]:
# Fazendo backup
bkp = enem_2019

In [13]:
# Aplicando alteração:
enem_2019 = enem_2019.dropna(how='any',subset=["NU_NOTA_CN",
                                   "NU_NOTA_CH",
                                   "NU_NOTA_LC",
                                   "NU_NOTA_MT",
                                   "NU_NOTA_REDACAO"])

In [14]:
# Validando a alteração: verificando se existem valores nulos nas colunas especificadas
enem_2019.filter((enem_2019["NU_NOTA_CN"].isNull()) |
                 (enem_2019["NU_NOTA_CH"].isNull()) |
                 (enem_2019["NU_NOTA_LC"].isNull()) |
                 (enem_2019["NU_NOTA_MT"].isNull()) |
                 (enem_2019["NU_NOTA_REDACAO"].isNull())).show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+----------------+---------+---------+----------------------+------------------+---------------+------------------+------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+---------------+---------------+---------------+---------------+---------+--------------+--------------+--------------+--------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_ANO_CONCLUIU|TP_ESCOLA|TP_ENSINO|IN_TREINEIRO|

In [15]:
# Filtrando colunas que não interessam para esta analise:
enem_2019.drop("TP_ANO_CONCLUIU",
               "TP_ST_CONCLUSAO",
               "CO_MUNICIPIO_ESC",
               "CO_UF_ESC",
               "CO_MUNICIPIO_PROVA",
               "NO_MUNICIPIO_ESC",
               "TP_SIT_FUNC_ESC",
               "TP_ENSINO",
               "TX_RESPOSTAS_CN",
               "TX_RESPOSTAS_CH",
               "TX_RESPOSTAS_LC",
               "TX_RESPOSTAS_MT",
               "TP_LINGUA",
               "TX_GABARITO_CN",
               "TX_GABARITO_CH",
               "TX_GABARITO_LC",
               "TX_GABARITO_MT",
               "TP_STATUS_REDACAO",
               "NU_NOTA_COMP1",
               "NU_NOTA_COMP2",
               "NU_NOTA_COMP3",
               "NU_NOTA_COMP4",
               "NU_NOTA_COMP5",
               "CO_UF_PROVA",
               "SG_UF_PROVA",
               "TP_PRESENCA_CN",
               "TP_PRESENCA_CH",
               "TP_PRESENCA_LC",
               "TP_PRESENCA_MT",
               "CO_PROVA_CN",
               "CO_PROVA_CH",
               "CO_PROVA_LC",
               "CO_PROVA_MT",
               "Q001",
               "Q002",
               "Q003",
               "Q004",
               "Q005",
               "Q006",
               "Q007",
               "Q008",
               "Q009",
               "Q010",
               "Q011",
               "Q012",
               "Q013",
               "Q014",
               "Q015",
               "Q016",
               "Q017",
               "Q018",
               "Q019",
               "Q020",
               "Q021",
               "Q022",
               "Q023",
               "Q024",
               "Q025").show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------+------------+---------+----------------------+------------------+--------------------+----------+----------+----------+----------+---------------+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ESCOLA|IN_TREINEIRO|SG_UF_ESC|TP_DEPENDENCIA_ADM_ESC|TP_LOCALIZACAO_ESC|  NO_MUNICIPIO_PROVA|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|NU_NOTA_REDACAO|
+------------+------+---------------+-------+---------------+-----------+----------------+---------+------------+---------+----------------------+------------------+--------------------+----------+----------+----------+----------+---------------+
|190001592266|  2019|              2|      M|              1|          1|               1|        2|           0|     NULL|                  NULL|              NULL|           S�o Paulo|     543.9|     548.1|     502.5|     480.7|            400|
|19000142154

In [16]:
# Aplicando Data frame
enem_2019 = enem_2019.drop(
               "TP_ANO_CONCLUIU",
               "TP_ESTADO_CIVIL",
               "TP_ST_CONCLUSAO",
               "CO_MUNICIPIO_ESC",
               "CO_UF_ESC",
               "CO_MUNICIPIO_PROVA",
               "NO_MUNICIPIO_ESC",
               "TP_SIT_FUNC_ESC",
               "TP_ENSINO",
               "TX_RESPOSTAS_CN",
               "TX_RESPOSTAS_CH",
               "TX_RESPOSTAS_LC",
               "TX_RESPOSTAS_MT",
               "TP_LINGUA",
               "TX_GABARITO_CN",
               "TX_GABARITO_CH",
               "TX_GABARITO_LC",
               "TX_GABARITO_MT",
               "TP_STATUS_REDACAO",
               "NU_NOTA_COMP1",
               "NU_NOTA_COMP2",
               "NU_NOTA_COMP3",
               "NU_NOTA_COMP4",
               "NU_NOTA_COMP5",
               "CO_UF_PROVA",
               "SG_UF_PROVA",
               "TP_PRESENCA_CN",
               "TP_PRESENCA_CH",
               "TP_PRESENCA_LC",
               "TP_PRESENCA_MT",
               "CO_PROVA_CN",
               "CO_PROVA_CH",
               "CO_PROVA_LC",
               "CO_PROVA_MT",
               "Q001",
               "Q002",
               "Q003",
               "Q004",
               "Q005",
               "Q006",
               "Q007",
               "Q008",
               "Q009",
               "Q010",
               "Q011",
               "Q012",
               "Q013",
               "Q014",
               "Q015",
               "Q016",
               "Q017",
               "Q018",
               "Q019",
               "Q020",
               "Q021",
               "Q022",
               "Q023",
               "Q024",
               "Q025")

In [17]:
enem_2019.show()

+------------+------+---------------+-------+-----------+----------------+---------+------------+---------+----------------------+------------------+--------------------+----------+----------+----------+----------+---------------+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_COR_RACA|TP_NACIONALIDADE|TP_ESCOLA|IN_TREINEIRO|SG_UF_ESC|TP_DEPENDENCIA_ADM_ESC|TP_LOCALIZACAO_ESC|  NO_MUNICIPIO_PROVA|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|NU_NOTA_REDACAO|
+------------+------+---------------+-------+-----------+----------------+---------+------------+---------+----------------------+------------------+--------------------+----------+----------+----------+----------+---------------+
|190001592266|  2019|              2|      M|          1|               1|        2|           0|     NULL|                  NULL|              NULL|           S�o Paulo|     543.9|     548.1|     502.5|     480.7|            400|
|190001421547|  2019|              4|      M|          3|               1|  

In [18]:
# Renomenando colunas:
enem_2019.withColumnsRenamed({
    "NU_INSCRICAO":"inscricao",
    "NU_ANO":"ano_prova",
    "TP_FAIXA_ETARIA":"faixa_etaria",
    "TP_SEXO":"genero",
    "TP_COR_RACA":"cor_raca",
    "TP_NACIONALIDADE":"nacionalidade",
    "TP_ESCOLA": "tipo_escola",
    "IN_TREINEIRO": "treineiro",
    "SG_UF_ESC":"uf_escola",
    "TP_DEPENDENCIA_ADM_ESC":"dependencia_esc",
    "TP_LOCALIZACAO_ESC":"localizacao_esc",
    "NO_MUNICIPIO_PROVA":"municipio_esc",
    "NU_NOTA_CN":"n_ciencias_nat",
    "NU_NOTA_CH":"n_ciencias_hum",
    "NU_NOTA_LC":"n_linguagens_cod",
    "NU_NOTA_MT":"n_matematica",
    "NU_NOTA_REDACAO":"n_redacao"
}).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|           2|     M|       1|            1|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|           4|     M|       3|            1|          1|        1|     NULL|         

In [19]:
# Aplicando alterações
enem_2019=enem_2019.withColumnsRenamed({
    "NU_INSCRICAO":"inscricao",
    "NU_ANO":"ano_prova",
    "TP_FAIXA_ETARIA":"faixa_etaria",
    "TP_SEXO":"genero",
    "TP_COR_RACA":"cor_raca",
    "TP_NACIONALIDADE":"nacionalidade",
    "TP_ESCOLA": "tipo_escola",
    "IN_TREINEIRO": "treineiro",
    "SG_UF_ESC":"uf_escola",
    "TP_DEPENDENCIA_ADM_ESC":"dependencia_esc",
    "TP_LOCALIZACAO_ESC":"localizacao_esc",
    "NO_MUNICIPIO_PROVA":"municipio_esc",
    "NU_NOTA_CN":"n_ciencias_nat",
    "NU_NOTA_CH":"n_ciencias_hum",
    "NU_NOTA_LC":"n_linguagens_cod",
    "NU_NOTA_MT":"n_matematica",
    "NU_NOTA_REDACAO":"n_redacao"
})

In [20]:
enem_2019.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|           2|     M|       1|            1|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|           4|     M|       3|            1|          1|        1|     NULL|         

In [21]:
# Alterando valores das colunas segundo dicionario de dados
enem_2019.withColumn(    "faixa_etaria",
                         f.when(enem_2019.faixa_etaria == 1, "MENOR QUE 17")\
                         .when(enem_2019.faixa_etaria == 2, "17")\
                         .when(enem_2019.faixa_etaria == 3, "18")\
                         .when(enem_2019.faixa_etaria == 4, "19")\
                         .when(enem_2019.faixa_etaria == 5, "20")\
                         .when(enem_2019.faixa_etaria == 6, "21")\
                         .when(enem_2019.faixa_etaria == 7, "22")\
                         .when(enem_2019.faixa_etaria == 8, "23")\
                         .when(enem_2019.faixa_etaria == 9, "24")\
                         .when(enem_2019.faixa_etaria == 10, "25").otherwise("N/I")
                         ).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|          17|     M|       1|            1|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|          19|     M|       3|            1|          1|        1|     NULL|         

In [22]:
# Implementando alteração:
enem_2019 = enem_2019.withColumn(    "faixa_etaria",
                         f.when(enem_2019.faixa_etaria == 1, "MENOR QUE 17")\
                         .when(enem_2019.faixa_etaria == 2, "17")\
                         .when(enem_2019.faixa_etaria == 3, "18")\
                         .when(enem_2019.faixa_etaria == 4, "19")\
                         .when(enem_2019.faixa_etaria == 5, "20")\
                         .when(enem_2019.faixa_etaria == 6, "21")\
                         .when(enem_2019.faixa_etaria == 7, "22")\
                         .when(enem_2019.faixa_etaria == 8, "23")\
                         .when(enem_2019.faixa_etaria == 9, "24")\
                         .when(enem_2019.faixa_etaria == 10, "25").otherwise("N/I")
                         )

In [23]:
enem_2019.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|          17|     M|       1|            1|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|          19|     M|       3|            1|          1|        1|     NULL|         

In [24]:
# Alterando valores das colunas segundo dicionario de dados
enem_2019.withColumn("cor_raca",
                         f.when(enem_2019.cor_raca == 1, "BRANCA")\
                         .when(enem_2019.cor_raca == 2, "PRETA")\
                         .when(enem_2019.cor_raca == 3, "PARDA")\
                         .when(enem_2019.cor_raca == 4, "AMARELA")
                         .when(enem_2019.cor_raca == 5, "INDIGENA").otherwise("N/I")
                         ).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|          17|     M|  BRANCA|            1|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|          19|     M|   PARDA|            1|          1|        1|     NULL|         

In [25]:
# Implementando alteração
enem_2019 = enem_2019.withColumn("cor_raca",
                         f.when(enem_2019.cor_raca == 1, "BRANCA")\
                         .when(enem_2019.cor_raca == 2, "PRETA")\
                         .when(enem_2019.cor_raca == 3, "PARDA")\
                         .when(enem_2019.cor_raca == 4, "AMARELA")
                         .when(enem_2019.cor_raca == 5, "INDIGENA").otherwise("N/I")
                         )

In [26]:
enem_2019.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|          17|     M|  BRANCA|            1|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|          19|     M|   PARDA|            1|          1|        1|     NULL|         

In [27]:
# Alterando valores de colunas segundo dicionario
enem_2019.withColumn("nacionalidade",
                         f.when(enem_2019.nacionalidade == 1, "BRASILEIRO(A)")\
                         .when(enem_2019.nacionalidade == 2, "BRASILEIRO(A) NATURALIZADO")\
                         .when(enem_2019.nacionalidade == 3, "ESTRANGEIRO")\
                         .when(enem_2019.nacionalidade == 4, "BRASILEIRO, NASCIDO NO EXTERIOR").otherwise("N/I")
                         ).show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|          17|     M|  BRANCA|       BRASILEIRO(A)|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|          19|     M|   PARDA|       BRASILEIRO(A)|      

In [28]:
# Implementando normalização:
enem_2019 = enem_2019.withColumn("nacionalidade",
                         f.when(enem_2019.nacionalidade == 1, "BRASILEIRO(A)")\
                         .when(enem_2019.nacionalidade == 2, "BRASILEIRO(A) NATURALIZADO")\
                         .when(enem_2019.nacionalidade == 3, "ESTRANGEIRO")\
                         .when(enem_2019.nacionalidade == 4, "BRASILEIRO, NASCIDO NO EXTERIOR").otherwise("N/I")
                         )

In [29]:
enem_2019.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|          17|     M|  BRANCA|       BRASILEIRO(A)|          2|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|          19|     M|   PARDA|       BRASILEIRO(A)|      

In [30]:
# Alterando valores das colunas segundo dicionario:
enem_2019.withColumn("tipo_escola",
                         f.when(enem_2019.tipo_escola == 2, "PUBLICA")\
                         .when(enem_2019.tipo_escola == 3, "PRIVADA")\
                         .when(enem_2019.tipo_escola == 4, "EXTERIOR")
).show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|          17|     M|  BRANCA|       BRASILEIRO(A)|    PUBLICA|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|          19|     M|   PARDA|       BRASILEIRO(A)|      

In [31]:
enem_2019 = enem_2019.withColumn("tipo_escola",
                         f.when(enem_2019.tipo_escola == 2, "PUBLICA")\
                         .when(enem_2019.tipo_escola == 3, "PRIVADA")\
                         .when(enem_2019.tipo_escola == 4, "EXTERIOR")
)

In [32]:
enem_2019.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|          17|     M|  BRANCA|       BRASILEIRO(A)|    PUBLICA|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001421547|     2019|          19|     M|   PARDA|       BRASILEIRO(A)|      

In [33]:
# Removendo valores nulos do tipo escola:
enem_2019.dropna(subset=["tipo_escola"]).show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|          17|     M|  BRANCA|       BRASILEIRO(A)|    PUBLICA|        0|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001867757|     2019|          17|     F|  BRANCA|       BRASILEIRO(A)|    PR

In [34]:
# Aplicando alteração:
enem_2019 = enem_2019.dropna(subset=["tipo_escola"])

In [35]:
# Verificando alteração:
enem_2019.filter((enem_2019["tipo_escola"].isNull())).show()

+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+
|inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+
+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+



In [36]:
# Alterando dados segundo dicionario
enem_2019.withColumn("treineiro",
                         f.when(enem_2019.treineiro == 1, "SIM").otherwise("NAO")
                         ).show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001592266|     2019|          17|     M|  BRANCA|       BRASILEIRO(A)|    PUBLICA|      NAO|     NULL|           NULL|           NULL|           S�o Paulo|         543.9|         548.1|           502.5|       480.7|      400|
|190001867757|     2019|          17|     F|  BRANCA|       BRASILEIRO(A)|    PR

In [37]:
enem_2019=enem_2019.withColumn("treineiro",
                         f.when(enem_2019.treineiro == 1, "SIM").otherwise("NAO"))

In [38]:
# Atualizando backup
bpk = enem_2019

In [39]:
# Tratando dados "uf_escola" e "municipio_esc" - se "uf_escola" e "municipio_esc"
# for nulo remova a linha
enem_2019.dropna(thresh=2,subset=["uf_escola","municipio_esc"]).show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001867757|     2019|          17|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|              4|              1|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|
|190001692704|     2019|          18|     F|  BRANCA|       BRASILEIRO(A)|    PU

In [40]:
enem_2019 = enem_2019.dropna(thresh=2,subset=["uf_escola","municipio_esc"])

In [41]:
enem_2019.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001867757|     2019|          17|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|              4|              1|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|
|190001692704|     2019|          18|     F|  BRANCA|       BRASILEIRO(A)|    PU

In [42]:
# Alterando dados de colunas segundo dicionario:
enem_2019.withColumn("dependencia_esc",
                         f.when(enem_2019.dependencia_esc == 1, "FEDERAL")\
                         .when(enem_2019.dependencia_esc == 2, "MUNICIPAL")\
                         .when(enem_2019.dependencia_esc == 3, "PRIVADA").otherwise("N/I")
).show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001867757|     2019|          17|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|            N/I|              1|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|
|190001692704|     2019|          18|     F|  BRANCA|       BRASILEIRO(A)|    PU

In [43]:
enem_2019 = enem_2019.withColumn("dependencia_esc",
                         f.when(enem_2019.dependencia_esc == 1, "FEDERAL")\
                         .when(enem_2019.dependencia_esc == 2, "MUNICIPAL")\
                         .when(enem_2019.dependencia_esc == 3, "PRIVADA").otherwise("N/I")
)

In [44]:
enem_2019.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001867757|     2019|          17|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|            N/I|              1|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|
|190001692704|     2019|          18|     F|  BRANCA|       BRASILEIRO(A)|    PU

In [45]:
# Alterando dados de colunas segundo dicionario:
enem_2019.withColumn("localizacao_esc",
                         f.when(enem_2019.localizacao_esc == 1, "URBANO")\
                         .when(enem_2019.localizacao_esc == 2, "RURAL")
).show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001867757|     2019|          17|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|            N/I|         URBANO|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|
|190001692704|     2019|          18|     F|  BRANCA|       BRASILEIRO(A)|    PU

In [46]:
enem_2019 = enem_2019.withColumn("localizacao_esc",
                         f.when(enem_2019.localizacao_esc == 1, "URBANO")\
                         .when(enem_2019.localizacao_esc == 2, "RURAL")
)

In [47]:
enem_2019.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+
|190001867757|     2019|          17|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|            N/I|         URBANO|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|
|190001692704|     2019|          18|     F|  BRANCA|       BRASILEIRO(A)|    PU

In [48]:
bkp= enem_2019

In [49]:
# Removendo dados nulos da coluna "localizacao_esc":
enem_2019 = enem_2019.dropna(subset=['localizacao_esc'])

In [50]:
# Verificando alteração
enem_2019.filter((enem_2019['localizacao_esc'].isNull())).show()

+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+
|inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+
+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+



In [51]:
bkp = enem_2019

In [52]:
# Adicionando a coluna media final para enem:
enem_2019 = enem_2019.withColumn("nota_final",
                                  (f.col("n_ciencias_nat") +
                                   f.col("n_ciencias_hum") +
                                   f.col("n_linguagens_cod") +
                                   f.col("n_matematica") +
                                   f.col("n_redacao")) / 5)

In [53]:
enem_2019.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+------------------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|        nota_final|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+------------------+
|190001867757|     2019|          17|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|            N/I|         URBANO|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|            722.58|
|190

In [54]:
bkp = enem_2019

In [55]:
# normalizando coluna nota final deixando 2 casa apos o . :
enem_2019 = enem_2019.withColumn("nota_final",
                                 f.format_number("nota_final", 2))

In [56]:
enem_2019.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+----------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|nota_final|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+----------+
|190001867757|     2019|          17|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|            N/I|         URBANO|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|    722.58|
|190001692704|     2019|          18



---

Realizando consultas SQL enem_2019:


---




In [57]:
enem_2019.count()

940926

In [58]:
# Criando view
enem_2019.createOrReplaceTempView("v_enem_2019")

In [59]:
spark.sql(
"""
SELECT A.uf_escola, ROUND(AVG(A.nota_final), 2) AS media_estado
FROM v_enem_2019 A
GROUP BY uf_escola
ORDER BY media_estado DESC
"""
).toPandas()

,uf_escola,media_estado
0,DF,542.84
1,RJ,542.60
2,RS,541.38
3,MG,540.90
4,SC,539.17
5,SP,538.54
6,ES,537.31
7,PR,533.68
8,GO,528.88
9,RN,526.97


In [60]:
enem_2019.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+----------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|nota_final|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+----------+
|190001867757|     2019|          17|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|            N/I|         URBANO|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|    722.58|
|190001692704|     2019|          18

In [61]:
spark.sql(
"""
SELECT A.municipio_esc,
COUNT(inscricao) AS total_inscricoes,
ROUND(AVG(nota_final), 2) as media_nota
FROM v_enem_2019 AS A
GROUP BY municipio_esc
ORDER BY total_inscricoes DESC, media_nota DESC
"""
).toPandas()

,municipio_esc,total_inscricoes,media_nota
0,S�o Paulo,41332,542.73
1,Rio de Janeiro,23575,561.74
2,Fortaleza,22659,523.71
3,Bras�lia,18145,542.97
4,Manaus,12909,498.41
...,...,...,...
1687,Porto Walter,19,489.35
1688,Canutama,19,463.26
1689,Fernando de Noronha,10,519.08
1690,Santa Rosa do Purus,5,482.14


In [62]:
spark.sql(
"""
SELECT A.uf_escola,
COUNT(inscricao) AS total_inscricoes,
ROUND(AVG(nota_final), 2) as media_nota
FROM v_enem_2019 AS A
GROUP BY uf_escola
ORDER BY total_inscricoes DESC, media_nota DESC
"""
).toPandas()

,uf_escola,total_inscricoes,media_nota
0,SP,175880,538.54
1,MG,90767,540.90
2,CE,86267,494.21
3,RJ,60938,542.60
4,BA,58818,507.92
5,PE,50617,511.08
6,PR,42171,533.68
7,PA,39039,500.12
8,RS,38669,541.38
9,MA,35959,484.08


In [63]:
spark.sql(
    """
    SELECT A.cor_raca,
           COUNT(A.inscricao) AS total_inscricoes,
           ROUND(AVG(nota_final), 2) as media_final
    FROM v_enem_2019 AS A
    GROUP BY A.cor_raca
    ORDER BY total_inscricoes DESC
    """
).toPandas()

,cor_raca,total_inscricoes,media_final
0,PARDA,426456,505.25
1,BRANCA,362858,549.57
2,PRETA,105626,502.04
3,AMARELA,21131,520.92
4,N/I,19275,512.64
5,INDIGENA,5580,479.79


In [64]:
spark.sql(
    """
SELECT A.inscricao AS numero_inscricao,
       A.municipio_esc AS municipio,
       A.n_ciencias_nat AS ciencias_naturais,
       A.n_ciencias_hum AS ciencias_humanas,
       A.n_linguagens_cod AS linguagens_codigos,
       A.n_matematica AS matematica,
       A.n_redacao AS redacao,
       A.nota_final
FROM v_enem_2019 AS A
WHERE nota_final > 100
ORDER BY nota_final DESC
    """
).toPandas()

,numero_inscricao,municipio,ciencias_naturais,ciencias_humanas,linguagens_codigos,matematica,redacao,nota_final
0,190003517496,Belo Horizonte,772.3,768.3,749.3,984.2,980,850.82
1,190002821921,Pouso Alegre,842.8,733.4,690.9,984.2,980,846.26
2,190001099380,Teresina,843.2,789.9,627.7,984.2,980,845.00
3,190002879782,Goi�nia,813.1,767.4,738.6,914.4,980,842.70
4,190004127950,Goi�nia,829.4,752.1,657.1,984.2,980,840.56
...,...,...,...,...,...,...,...,...
940917,190001158964,Quiterian�polis,363.3,0.0,0.0,376.0,0,147.86
940918,190003014626,Cuiab�,404.3,0.0,322.0,0.0,0,145.26
940919,190001366880,Recife,384.5,0.0,324.0,0.0,0,141.70
940920,190003609385,Juazeiro do Norte,0.0,0.0,0.0,0.0,620,124.00


---





**Importando e iniciando tratamento de dados Data Frame enem_2020**

---




In [65]:
# Importando a base de dados do enem de 2020
url = "/content/drive/MyDrive/BasesSpark/enem/enem_2020/DADOS/MICRODADOS*"
enem_2020 =spark.read.csv(
    url,
    header=True,
    inferSchema=True,
    encoding= "UTF-8",
    sep=";"
)

In [66]:
enem_2020.show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|

In [67]:
enem_2020.printSchema()

root
 |-- NU_INSCRICAO: long (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- TP_FAIXA_ETARIA: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: integer (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- TP_ST_CONCLUSAO: integer (nullable = true)
 |-- TP_ANO_CONCLUIU: integer (nullable = true)
 |-- TP_ESCOLA: integer (nullable = true)
 |-- TP_ENSINO: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_MUNICIPIO_ESC: integer (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: integer (nullable = true)
 |-- TP_LOCALIZACAO_ESC: integer (nullable = true)
 |-- TP_SIT_FUNC_ESC: integer (nullable = true)
 |-- CO_MUNICIPIO_PROVA: integer (nullable = true)
 |-- NO_MUNICIPIO_PROVA: string (nullable = true)
 |-- CO_UF_PROVA: integer 

In [68]:
enem_2020.count()

5783109

In [69]:
# Filtrando alunos que estiveram presentes em todos os dias de prova, que nao concluiram ou irão concluir o ensino medio
enem_2020.where(
    "TP_FAIXA_ETARIA <= 10 and TP_PRESENCA_CN like '1' and TP_PRESENCA_CH like '1' and TP_PRESENCA_LC like '1' and TP_PRESENCA_MT like '1' and TP_ST_CONCLUSAO like '2' or TP_ST_CONCLUSAO like '3'"
).show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+-------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|T

In [70]:
enem_2020 =enem_2020.where(
    "TP_FAIXA_ETARIA <= 10 and TP_PRESENCA_CN like '1' and TP_PRESENCA_CH like '1' and TP_PRESENCA_LC like '1' and TP_PRESENCA_MT like '1' and TP_ST_CONCLUSAO like '2' or TP_ST_CONCLUSAO like '3'"
)

In [71]:
# Removendo notas nulas:
enem_2020.dropna(how='any',subset=["NU_NOTA_CN",
                                   "NU_NOTA_CH",
                                   "NU_NOTA_LC",
                                   "NU_NOTA_MT",
                                   "NU_NOTA_REDACAO"]).show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+-------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|T

In [72]:
# Removendo notas nulas:
enem_2020= enem_2020.dropna(how='any',subset=["NU_NOTA_CN",
                                   "NU_NOTA_CH",
                                   "NU_NOTA_LC",
                                   "NU_NOTA_MT",
                                   "NU_NOTA_REDACAO"])

In [73]:
# Validando a alteração: verificando se existem valores nulos nas colunas especificadas
enem_2020.filter((enem_2020["NU_NOTA_CN"].isNull()) |
                 (enem_2020["NU_NOTA_CH"].isNull()) |
                 (enem_2020["NU_NOTA_LC"].isNull()) |
                 (enem_2020["NU_NOTA_MT"].isNull()) |
                 (enem_2020["NU_NOTA_REDACAO"].isNull())).show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+----------------+---------+---------+----------------------+------------------+---------------+------------------+------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+---------------+---------------+---------------+---------------+---------+--------------+--------------+--------------+--------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_ANO_CONCLUIU|TP_ESCOLA|TP_ENSINO|IN_TREINEIRO|

In [74]:
# Filtrando colunas que não interessam para esta analise:
enem_2020.drop("TP_ANO_CONCLUIU",
               "TP_ST_CONCLUSAO",
               "CO_MUNICIPIO_ESC",
               "CO_UF_ESC",
               "CO_MUNICIPIO_PROVA",
               "NO_MUNICIPIO_ESC",
               "TP_SIT_FUNC_ESC",
               "TP_ENSINO",
               "TX_RESPOSTAS_CN",
               "TX_RESPOSTAS_CH",
               "TX_RESPOSTAS_LC",
               "TX_RESPOSTAS_MT",
               "TP_LINGUA",
               "TX_GABARITO_CN",
               "TX_GABARITO_CH",
               "TX_GABARITO_LC",
               "TX_GABARITO_MT",
               "TP_STATUS_REDACAO",
               "NU_NOTA_COMP1",
               "NU_NOTA_COMP2",
               "NU_NOTA_COMP3",
               "NU_NOTA_COMP4",
               "NU_NOTA_COMP5",
               "CO_UF_PROVA",
               "SG_UF_PROVA",
               "TP_PRESENCA_CN",
               "TP_PRESENCA_CH",
               "TP_PRESENCA_LC",
               "TP_PRESENCA_MT",
               "CO_PROVA_CN",
               "CO_PROVA_CH",
               "CO_PROVA_LC",
               "CO_PROVA_MT",
               "Q001",
               "Q002",
               "Q003",
               "Q004",
               "Q005",
               "Q006",
               "Q007",
               "Q008",
               "Q009",
               "Q010",
               "Q011",
               "Q012",
               "Q013",
               "Q014",
               "Q015",
               "Q016",
               "Q017",
               "Q018",
               "Q019",
               "Q020",
               "Q021",
               "Q022",
               "Q023",
               "Q024",
               "Q025").show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------+------------+---------+----------------------+------------------+-------------------+----------+----------+----------+----------+---------------+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ESCOLA|IN_TREINEIRO|SG_UF_ESC|TP_DEPENDENCIA_ADM_ESC|TP_LOCALIZACAO_ESC| NO_MUNICIPIO_PROVA|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|NU_NOTA_REDACAO|
+------------+------+---------------+-------+---------------+-----------+----------------+---------+------------+---------+----------------------+------------------+-------------------+----------+----------+----------+----------+---------------+
|200001908998|  2020|              2|      M|              1|          3|               1|        2|           0|       SP|                     3|                 1|Santana de Parna�ba|     620.8|     675.0|     624.2|     759.4|            760|
|200003209996|  

In [75]:
enem_2020 = enem_2020.drop(
               "TP_ANO_CONCLUIU",
               "TP_ESTADO_CIVIL",
               "TP_ST_CONCLUSAO",
               "CO_MUNICIPIO_ESC",
               "CO_UF_ESC",
               "CO_MUNICIPIO_PROVA",
               "NO_MUNICIPIO_ESC",
               "TP_SIT_FUNC_ESC",
               "TP_ENSINO",
               "TX_RESPOSTAS_CN",
               "TX_RESPOSTAS_CH",
               "TX_RESPOSTAS_LC",
               "TX_RESPOSTAS_MT",
               "TP_LINGUA",
               "TX_GABARITO_CN",
               "TX_GABARITO_CH",
               "TX_GABARITO_LC",
               "TX_GABARITO_MT",
               "TP_STATUS_REDACAO",
               "NU_NOTA_COMP1",
               "NU_NOTA_COMP2",
               "NU_NOTA_COMP3",
               "NU_NOTA_COMP4",
               "NU_NOTA_COMP5",
               "CO_UF_PROVA",
               "SG_UF_PROVA",
               "TP_PRESENCA_CN",
               "TP_PRESENCA_CH",
               "TP_PRESENCA_LC",
               "TP_PRESENCA_MT",
               "CO_PROVA_CN",
               "CO_PROVA_CH",
               "CO_PROVA_LC",
               "CO_PROVA_MT",
               "Q001",
               "Q002",
               "Q003",
               "Q004",
               "Q005",
               "Q006",
               "Q007",
               "Q008",
               "Q009",
               "Q010",
               "Q011",
               "Q012",
               "Q013",
               "Q014",
               "Q015",
               "Q016",
               "Q017",
               "Q018",
               "Q019",
               "Q020",
               "Q021",
               "Q022",
               "Q023",
               "Q024",
               "Q025")

In [76]:
enem_2020.show()

+------------+------+---------------+-------+-----------+----------------+---------+------------+---------+----------------------+------------------+-------------------+----------+----------+----------+----------+---------------+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_COR_RACA|TP_NACIONALIDADE|TP_ESCOLA|IN_TREINEIRO|SG_UF_ESC|TP_DEPENDENCIA_ADM_ESC|TP_LOCALIZACAO_ESC| NO_MUNICIPIO_PROVA|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|NU_NOTA_REDACAO|
+------------+------+---------------+-------+-----------+----------------+---------+------------+---------+----------------------+------------------+-------------------+----------+----------+----------+----------+---------------+
|200001908998|  2020|              2|      M|          3|               1|        2|           0|       SP|                     3|                 1|Santana de Parna�ba|     620.8|     675.0|     624.2|     759.4|            760|
|200003209996|  2020|              3|      F|          3|               1|      

In [77]:
# Renomenando colunas:
enem_2020.withColumnsRenamed({
    "NU_INSCRICAO":"inscricao",
    "NU_ANO":"ano_prova",
    "TP_FAIXA_ETARIA":"faixa_etaria",
    "TP_SEXO":"genero",
    "TP_COR_RACA":"cor_raca",
    "TP_NACIONALIDADE":"nacionalidade",
    "TP_ESCOLA": "tipo_escola",
    "IN_TREINEIRO": "treineiro",
    "SG_UF_ESC":"uf_escola",
    "TP_DEPENDENCIA_ADM_ESC":"dependencia_esc",
    "TP_LOCALIZACAO_ESC":"localizacao_esc",
    "NO_MUNICIPIO_PROVA":"municipio_esc",
    "NU_NOTA_CN":"n_ciencias_nat",
    "NU_NOTA_CH":"n_ciencias_hum",
    "NU_NOTA_LC":"n_linguagens_cod",
    "NU_NOTA_MT":"n_matematica",
    "NU_NOTA_REDACAO":"n_redacao"
}).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|           2|     M|       3|            1|          2|        0|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|           3|     F|       3|            1|          2|        0|       AM|             

In [78]:
# Aplicando alterações
enem_2020=enem_2020.withColumnsRenamed({
    "NU_INSCRICAO":"inscricao",
    "NU_ANO":"ano_prova",
    "TP_FAIXA_ETARIA":"faixa_etaria",
    "TP_SEXO":"genero",
    "TP_COR_RACA":"cor_raca",
    "TP_NACIONALIDADE":"nacionalidade",
    "TP_ESCOLA": "tipo_escola",
    "IN_TREINEIRO": "treineiro",
    "SG_UF_ESC":"uf_escola",
    "TP_DEPENDENCIA_ADM_ESC":"dependencia_esc",
    "TP_LOCALIZACAO_ESC":"localizacao_esc",
    "NO_MUNICIPIO_PROVA":"municipio_esc",
    "NU_NOTA_CN":"n_ciencias_nat",
    "NU_NOTA_CH":"n_ciencias_hum",
    "NU_NOTA_LC":"n_linguagens_cod",
    "NU_NOTA_MT":"n_matematica",
    "NU_NOTA_REDACAO":"n_redacao"
})

In [79]:
enem_2020.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|           2|     M|       3|            1|          2|        0|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|           3|     F|       3|            1|          2|        0|       AM|             

In [80]:
# Alterando valores das colunas segundo dicionario de dados
enem_2020.withColumn(    "faixa_etaria",
                         f.when(enem_2020.faixa_etaria == 1, "MENOR QUE 17")\
                         .when(enem_2020.faixa_etaria == 2, "17")\
                         .when(enem_2020.faixa_etaria == 3, "18")\
                         .when(enem_2020.faixa_etaria == 4, "19")\
                         .when(enem_2020.faixa_etaria == 5, "20")\
                         .when(enem_2020.faixa_etaria == 6, "21")\
                         .when(enem_2020.faixa_etaria == 7, "22")\
                         .when(enem_2020.faixa_etaria == 8, "23")\
                         .when(enem_2020.faixa_etaria == 9, "24")\
                         .when(enem_2020.faixa_etaria == 10, "25").otherwise("N/I")
                         ).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|          17|     M|       3|            1|          2|        0|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|          18|     F|       3|            1|          2|        0|       AM|             

In [81]:
# Aplicando valores
enem_2020 =  enem_2020.withColumn(    "faixa_etaria",
                         f.when(enem_2020.faixa_etaria == 1, "MENOR QUE 17")\
                         .when(enem_2020.faixa_etaria == 2, "17")\
                         .when(enem_2020.faixa_etaria == 3, "18")\
                         .when(enem_2020.faixa_etaria == 4, "19")\
                         .when(enem_2020.faixa_etaria == 5, "20")\
                         .when(enem_2020.faixa_etaria == 6, "21")\
                         .when(enem_2020.faixa_etaria == 7, "22")\
                         .when(enem_2020.faixa_etaria == 8, "23")\
                         .when(enem_2020.faixa_etaria == 9, "24")\
                         .when(enem_2020.faixa_etaria == 10, "25").otherwise("N/I")
                         )

In [82]:
enem_2020.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|          17|     M|       3|            1|          2|        0|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|          18|     F|       3|            1|          2|        0|       AM|             

In [83]:
# Alterando valores das colunas segundo dicionario de dados
enem_2020.withColumn("cor_raca",
                         f.when(enem_2020.cor_raca == 1, "BRANCA")\
                         .when(enem_2020.cor_raca == 2, "PRETA")\
                         .when(enem_2020.cor_raca == 3, "PARDA")\
                         .when(enem_2020.cor_raca == 4, "AMARELA")
                         .when(enem_2020.cor_raca == 5, "INDIGENA").otherwise("N/I")
                         ).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|          17|     M|   PARDA|            1|          2|        0|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|          18|     F|   PARDA|            1|          2|        0|       AM|             

In [84]:
# Alterando valores das colunas segundo dicionario de dados
enem_2020 = enem_2020.withColumn("cor_raca",
                         f.when(enem_2020.cor_raca == 1, "BRANCA")\
                         .when(enem_2020.cor_raca == 2, "PRETA")\
                         .when(enem_2020.cor_raca == 3, "PARDA")\
                         .when(enem_2020.cor_raca == 4, "AMARELA")
                         .when(enem_2020.cor_raca == 5, "INDIGENA").otherwise("N/I")
                         )

In [85]:
enem_2020.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|          17|     M|   PARDA|            1|          2|        0|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|          18|     F|   PARDA|            1|          2|        0|       AM|             

In [86]:
# Alterando valores de colunas segundo dicionario
enem_2020.withColumn("nacionalidade",
                         f.when(enem_2020.nacionalidade == 1, "BRASILEIRO(A)")\
                         .when(enem_2020.nacionalidade == 2, "BRASILEIRO(A) NATURALIZADO")\
                         .when(enem_2020.nacionalidade == 3, "ESTRANGEIRO")\
                         .when(enem_2020.nacionalidade == 4, "BRASILEIRO, NASCIDO NO EXTERIOR").otherwise("N/I")
                         ).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|          17|     M|   PARDA|BRASILEIRO(A)|          2|        0|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|          18|     F|   PARDA|BRASILEIRO(A)|          2|        0|       AM|             

In [87]:
# Aplicando alteração:
enem_2020=enem_2020.withColumn("nacionalidade",
                         f.when(enem_2020.nacionalidade == 1, "BRASILEIRO(A)")\
                         .when(enem_2020.nacionalidade == 2, "BRASILEIRO(A) NATURALIZADO")\
                         .when(enem_2020.nacionalidade == 3, "ESTRANGEIRO")\
                         .when(enem_2020.nacionalidade == 4, "BRASILEIRO, NASCIDO NO EXTERIOR").otherwise("N/I")
                         )

In [88]:
enem_2020.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|          17|     M|   PARDA|BRASILEIRO(A)|          2|        0|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|          18|     F|   PARDA|BRASILEIRO(A)|          2|        0|       AM|             

In [89]:
# Alterando valores das colunas segundo dicionario:
enem_2020.withColumn("tipo_escola",
                         f.when(enem_2020.tipo_escola == 2, "PUBLICA")\
                         .when(enem_2020.tipo_escola == 3, "PRIVADA")\
                         .when(enem_2020.tipo_escola == 4, "EXTERIOR")
).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|          17|     M|   PARDA|BRASILEIRO(A)|    PUBLICA|        0|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|          18|     F|   PARDA|BRASILEIRO(A)|    PUBLICA|        0|       AM|             

In [90]:
# Aplicando alterações:
enem_2020 = enem_2020.withColumn("tipo_escola",
                         f.when(enem_2020.tipo_escola == 2, "PUBLICA")\
                         .when(enem_2020.tipo_escola == 3, "PRIVADA")\
                         .when(enem_2020.tipo_escola == 4, "EXTERIOR")
)

In [91]:
enem_2020.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|          17|     M|   PARDA|BRASILEIRO(A)|    PUBLICA|        0|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|          18|     F|   PARDA|BRASILEIRO(A)|    PUBLICA|        0|       AM|             

In [92]:
# Removendo valores nulos do tipo escola:
enem_2020.dropna(subset=["tipo_escola"]).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|          17|     M|   PARDA|BRASILEIRO(A)|    PUBLICA|        0|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|          18|     F|   PARDA|BRASILEIRO(A)|    PUBLICA|        0|       AM|             

In [93]:
# Aplicando alteração
enem_2020 = enem_2020.dropna(subset=["tipo_escola"])

In [94]:
# Verificando alteração:
enem_2020.filter((enem_2020["tipo_escola"].isNull())).show()

+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+
|inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+
+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+



In [95]:
# Alterando dados segundo dicionario
enem_2020.withColumn("treineiro",
                         f.when(enem_2020.treineiro == 1, "SIM").otherwise("NAO")
                         ).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|          17|     M|   PARDA|BRASILEIRO(A)|    PUBLICA|      NAO|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|          18|     F|   PARDA|BRASILEIRO(A)|    PUBLICA|      NAO|       AM|             

In [96]:
enem_2020 = enem_2020.withColumn("treineiro",
                         f.when(enem_2020.treineiro == 1, "SIM").otherwise("NAO")
                         )

In [97]:
enem_2020.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|          17|     M|   PARDA|BRASILEIRO(A)|    PUBLICA|      NAO|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|          18|     F|   PARDA|BRASILEIRO(A)|    PUBLICA|      NAO|       AM|             

In [98]:
# Atualizand bkp
bkp = enem_2020

In [99]:
# Tratando dados "uf_escola" e "municipio_esc" - se "uf_escola" e "municipio_esc"
# for nulo remova a linha
enem_2020.dropna(thresh= 2, subset=["uf_escola", "municipio_esc"]).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|          17|     M|   PARDA|BRASILEIRO(A)|    PUBLICA|      NAO|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|          18|     F|   PARDA|BRASILEIRO(A)|    PUBLICA|      NAO|       AM|             

In [100]:
enem_2020 = enem_2020.dropna(thresh= 2, subset=["uf_escola", "municipio_esc"])

In [101]:
# Verificando alteração:
enem_2020.filter((enem_2020["uf_escola"].isNull()) |
                 (enem_2020["municipio_esc"].isNull())).show()

+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+
|inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+
+---------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------+--------------+--------------+----------------+------------+---------+



In [102]:
enem_2020.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|          17|     M|   PARDA|BRASILEIRO(A)|    PUBLICA|      NAO|       SP|              3|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|          18|     F|   PARDA|BRASILEIRO(A)|    PUBLICA|      NAO|       AM|             

In [103]:
# Alterando dados de colunas segundo dicionario:
enem_2020.withColumn("dependencia_esc",
                         f.when(enem_2020.dependencia_esc == 1, "FEDERAL")\
                         .when(enem_2020.dependencia_esc == 2, "MUNICIPAL")\
                         .when(enem_2020.dependencia_esc == 3, "PRIVADA").otherwise("N/I")
).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|          17|     M|   PARDA|BRASILEIRO(A)|    PUBLICA|      NAO|       SP|        PRIVADA|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|          18|     F|   PARDA|BRASILEIRO(A)|    PUBLICA|      NAO|       AM|      MUNICIP

In [104]:
# Aplicando alteração:
enem_2020 = enem_2020.withColumn("dependencia_esc",
                         f.when(enem_2020.dependencia_esc == 1, "FEDERAL")\
                         .when(enem_2020.dependencia_esc == 2, "MUNICIPAL")\
                         .when(enem_2020.dependencia_esc == 3, "PRIVADA").otherwise("N/I")
)

In [105]:
enem_2020.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|          17|     M|   PARDA|BRASILEIRO(A)|    PUBLICA|      NAO|       SP|        PRIVADA|              1|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|          18|     F|   PARDA|BRASILEIRO(A)|    PUBLICA|      NAO|       AM|      MUNICIP

In [106]:
# Alterando dados de colunas segundo dicionario:
enem_2020.withColumn("localizacao_esc",
                         f.when(enem_2020.localizacao_esc == 1, "URBANO")\
                         .when(enem_2020.localizacao_esc == 2, "RURAL")
).show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|          17|     M|   PARDA|BRASILEIRO(A)|    PUBLICA|      NAO|       SP|        PRIVADA|         URBANO|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|          18|     F|   PARDA|BRASILEIRO(A)|    PUBLICA|      NAO|       AM|      MUNICIP

In [107]:
# Aplicando alteração do produto:
enem_2020= enem_2020.withColumn("localizacao_esc",
                         f.when(enem_2020.localizacao_esc == 1, "URBANO")\
                         .when(enem_2020.localizacao_esc == 2, "RURAL"))

In [108]:
enem_2020.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+
|200001908998|     2020|          17|     M|   PARDA|BRASILEIRO(A)|    PUBLICA|      NAO|       SP|        PRIVADA|         URBANO|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|
|200003209996|     2020|          18|     F|   PARDA|BRASILEIRO(A)|    PUBLICA|      NAO|       AM|      MUNICIP

In [109]:
# Atualizando bkp
bkp = enem_2020

In [110]:
# Adicionando a coluna "media_final" para enem:
enem_2020 = enem_2020.withColumn("nota_final",
                                  (f.col("n_ciencias_nat") +
                                   f.col("n_ciencias_hum") +
                                   f.col("n_linguagens_cod") +
                                   f.col("n_matematica") +
                                   f.col("n_redacao")) / 5)

In [111]:
enem_2020.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+------------------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|        nota_final|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+------------------+
|200001908998|     2020|          17|     M|   PARDA|BRASILEIRO(A)|    PUBLICA|      NAO|       SP|        PRIVADA|         URBANO|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|            687.88|
|200003209996|     2020|          18

In [112]:
# Atualizando bkp:
bkp = enem_2020

In [113]:
# normalizando coluna nota final deixando 2 casa apos o . :
enem_2020 = enem_2020.withColumn("nota_final",
                                 f.format_number("nota_final", 2))

In [123]:
enem_2020.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+----------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|      municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|nota_final|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+-------------------+--------------+--------------+----------------+------------+---------+----------+
|200001908998|     2020|          17|     M|   PARDA|BRASILEIRO(A)|    PUBLICA|      NAO|       SP|        PRIVADA|         URBANO|Santana de Parna�ba|         620.8|         675.0|           624.2|       759.4|      760|    687.88|
|200003209996|     2020|          18|     F|   PARDA|BRASILEIRO(A)| 

In [132]:
# Criando view
enem_2020.createOrReplaceTempView("v_enem_2020")



---


**Iniciando consulta SQL enem_2020
**

---



In [133]:
# Media de nota por estado:
spark.sql(
    """
SELECT A.uf_escola,
       ROUND(AVG(A.nota_final), 2) AS media_estado
FROM v_enem_2020 A
GROUP BY A.uf_escola
ORDER BY media_estado DESC
""").toPandas()

,uf_escola,media_estado
0,MG,575.65
1,SP,564.92
2,RJ,564.73
3,SC,560.84
4,RS,557.65
5,PR,556.88
6,DF,554.49
7,ES,551.82
8,RN,550.70
9,GO,544.27


In [135]:
#Total de inscricores por uf:
spark.sql(
"""
SELECT A.uf_escola,
COUNT(inscricao) AS total_inscricoes,
ROUND(AVG(nota_final), 2) as media_nota
FROM v_enem_2020 AS A
GROUP BY uf_escola
ORDER BY total_inscricoes DESC, media_nota DESC
"""
).toPandas()

,uf_escola,total_inscricoes,media_nota
0,SP,98004,564.92
1,CE,51392,504.11
2,MG,46700,575.65
3,RJ,40189,564.73
4,BA,31648,528.25
5,PE,27405,530.39
6,PR,26704,556.88
7,RS,22923,557.65
8,PA,20885,511.95
9,GO,19726,544.27


In [136]:
# Analisando qnt de pessoas inscridas por "cor_raca" e sua "media_final":
spark.sql(
    """
    SELECT A.cor_raca,
           COUNT(A.inscricao) AS total_inscricoes,
           ROUND(AVG(nota_final), 2) as media_final
    FROM v_enem_2020 AS A
    GROUP BY A.cor_raca
    ORDER BY total_inscricoes DESC
    """
).toPandas()

,cor_raca,total_inscricoes,media_final
0,BRANCA,243867,572.67
1,PARDA,217364,518.28
2,PRETA,48344,512.82
3,AMARELA,11246,545.24
4,N/I,10701,536.17
5,INDIGENA,2438,480.77


In [137]:
# Analisando maiores medias por municipio
spark.sql(
    """
SELECT A.inscricao AS numero_inscricao,
       A.municipio_esc AS municipio,
       A.n_ciencias_nat AS ciencias_naturais,
       A.n_ciencias_hum AS ciencias_humanas,
       A.n_linguagens_cod AS linguagens_codigos,
       A.n_matematica AS matematica,
       A.n_redacao AS redacao,
       A.nota_final
FROM v_enem_2020 AS A
WHERE nota_final > 100
ORDER BY nota_final DESC
    """
).toPandas()

,numero_inscricao,municipio,ciencias_naturais,ciencias_humanas,linguagens_codigos,matematica,redacao,nota_final
0,200001357436,Rio de Janeiro,795.8,839.5,713.5,975.0,940,852.76
1,200001850224,S�o Paulo,800.8,862.6,716.9,896.4,980,851.34
2,200006051511,Jo�o Pessoa,756.7,819.2,725.4,952.4,1000,850.74
3,200005145247,Mogi das Cruzes,754.0,838.0,687.0,975.0,980,846.80
4,200001423299,Belo Horizonte,798.3,742.7,759.6,964.4,960,845.00
...,...,...,...,...,...,...,...,...
533951,200005639741,Momba�a,356.3,0.0,0.0,353.2,0,141.90
533952,200005819500,Catal�o,346.1,0.0,356.6,0.0,0,140.54
533953,200004244813,Pau dos Ferros,363.7,0.0,307.6,0.0,0,134.26
533954,200004878580,Salvador,351.6,0.0,314.6,0.0,0,133.24


In [138]:
enem_2020.count()

533960

In [139]:
enem_2019.count()

940926



---

Unindo dataframes tratados criando arquivo formato ORC e CSV:


---



In [147]:
# Unindo DataFrames tratados
enem_19_20 = enem_2019.union(enem_2020)

In [148]:
enem_19_20.show()

+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+----------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|       nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|nota_final|
+------------+---------+------------+------+--------+--------------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+----------+
|190001867757|     2019|          17|     F|  BRANCA|       BRASILEIRO(A)|    PRIVADA|      NAO|       PR|            N/I|         URBANO|           Apucarana|         618.2|         744.7|           636.3|       713.7|      900|    722.58|
|190001692704|     2019|          18

In [149]:
# Salvando dataframes / views em um arquivo.orc
url = "/content/drive/MyDrive/BasesSpark/enem/enem_19_20_ORC"
enem_19_20.write.orc(
    url,
    mode="overwrite"
)

In [150]:
# Salvando dataframes / views em um arquivo.csv
url = "/content/drive/MyDrive/BasesSpark/enem/enem_19_20_CSV"
enem_19_20.write.csv(
    url,
    mode="overwrite"
)



---



*   Importando enem_19_20.ORC
*   Realizando consultas SQL




---




In [151]:
# Importando arquivo orc:
url = "/content/drive/MyDrive/BasesSpark/enem/enem_19_20_ORC/part*"
enem_orc = spark.read.orc(url)

In [152]:
enem_orc.show()

+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+----------+
|   inscricao|ano_prova|faixa_etaria|genero|cor_raca|nacionalidade|tipo_escola|treineiro|uf_escola|dependencia_esc|localizacao_esc|       municipio_esc|n_ciencias_nat|n_ciencias_hum|n_linguagens_cod|n_matematica|n_redacao|nota_final|
+------------+---------+------------+------+--------+-------------+-----------+---------+---------+---------------+---------------+--------------------+--------------+--------------+----------------+------------+---------+----------+
|190001795646|     2019|          17|     F|   PARDA|BRASILEIRO(A)|    PUBLICA|      NAO|       SP|      MUNICIPAL|         URBANO|     Itaquaquecetuba|         422.8|         508.9|           558.6|       409.4|      520|    483.94|
|190001074908|     2019|          17|     F|   PRETA|BRASILEIRO(

In [153]:
# Criando view:
enem_orc.createOrReplaceTempView("v_enem_orc")

In [185]:
# Media de nota por estado:
spark.sql(
    """
SELECT A.ano_prova,
       A.uf_escola,
       ROUND(AVG(A.nota_final), 2) AS media_estado
FROM v_enem_orc A
GROUP BY A.uf_escola, A.ano_prova
ORDER BY uf_escola, media_estado DESC
""").toPandas()

,ano_prova,uf_escola,media_estado
0,2020,AC,507.62
1,2019,AC,501.45
2,2020,AL,520.80
3,2019,AL,500.26
4,2020,AM,498.16
5,2019,AM,486.38
6,2020,AP,499.06
7,2019,AP,491.57
8,2020,BA,528.25
9,2019,BA,507.92


In [188]:
#Total de inscricores por uf:
spark.sql(
"""
SELECT A.uf_escola,
       A.ano_prova,
COUNT(inscricao) AS total_inscricoes,
ROUND(AVG(nota_final), 2) as media_nota
FROM v_enem_orc AS A
GROUP BY ano_prova,
         uf_escola
ORDER BY uf_escola,
         total_inscricoes DESC,
         media_nota DESC
"""
).toPandas()

,uf_escola,ano_prova,total_inscricoes,media_nota
0,AC,2019,4538,501.45
1,AC,2020,2364,507.62
2,AL,2019,14632,500.26
3,AL,2020,8746,520.80
4,AM,2019,19795,486.38
5,AM,2020,10808,498.16
6,AP,2019,5061,491.57
7,AP,2020,2151,499.06
8,BA,2019,58818,507.92
9,BA,2020,31648,528.25


In [192]:
# Analisando qnt de pessoas inscridas por "cor_raca" e sua "media_final":
spark.sql(
    """
    SELECT A.ano_prova,
           A.cor_raca,
           COUNT(A.inscricao) AS total_inscricoes,
           ROUND(AVG(nota_final), 2) as media_final
    FROM v_enem_orc AS A
    GROUP BY A.ano_prova, A.cor_raca
    ORDER BY cor_raca,total_inscricoes DESC
    """
).toPandas()

,ano_prova,cor_raca,total_inscricoes,media_final
0,2019,AMARELA,21131,520.92
1,2020,AMARELA,11246,545.24
2,2019,BRANCA,362858,549.57
3,2020,BRANCA,243867,572.67
4,2019,INDIGENA,5580,479.79
5,2020,INDIGENA,2438,480.77
6,2019,N/I,19275,512.64
7,2020,N/I,10701,536.17
8,2019,PARDA,426456,505.25
9,2020,PARDA,217364,518.28


In [170]:
# Analisando maiores médias por município
spark.sql(
    """
    SELECT DISTINCT(A.ano_prova) AS ano_prova,
           A.inscricao AS numero_inscricao,
           A.municipio_esc AS municipio,
           A.n_ciencias_nat AS ciencias_naturais,
           A.n_ciencias_hum AS ciencias_humanas,
           A.n_linguagens_cod AS linguagens_codigos,
           A.n_matematica AS matematica,
           A.n_redacao AS redacao,
           A.nota_final
    FROM v_enem_orc AS A
    WHERE A.nota_final > 100
    ORDER BY A.nota_final DESC
    """
).limit(30).toPandas()

,ano_prova,numero_inscricao,municipio,ciencias_naturais,ciencias_humanas,linguagens_codigos,matematica,redacao,nota_final
0,2020,200001357436,Rio de Janeiro,795.8,839.5,713.5,975.0,940,852.76
1,2020,200001850224,S�o Paulo,800.8,862.6,716.9,896.4,980,851.34
2,2019,190003517496,Belo Horizonte,772.3,768.3,749.3,984.2,980,850.82
3,2020,200006051511,Jo�o Pessoa,756.7,819.2,725.4,952.4,1000,850.74
4,2020,200005145247,Mogi das Cruzes,754.0,838.0,687.0,975.0,980,846.80
5,2019,190002821921,Pouso Alegre,842.8,733.4,690.9,984.2,980,846.26
6,2020,200001423299,Belo Horizonte,798.3,742.7,759.6,964.4,960,845.00
7,2019,190001099380,Teresina,843.2,789.9,627.7,984.2,980,845.00
8,2020,200001622477,Salvador,808.2,818.2,707.7,910.4,980,844.90
9,2020,200005414259,Bras�lia,773.2,839.5,705.0,923.8,980,844.30


In [181]:
#  total de inscritos:
spark.sql(
    """
    SELECT A.ano_prova AS ano,
           COUNT(A.inscricao) AS total_inscritos
    FROM v_enem_orc A
    GROUP BY A.ano_prova
    ORDER BY ano, total_inscritos DESC
    """
).toPandas()

,ano,total_inscritos
0,2019,940926
1,2020,533960




---

Com esta breve análise sobre o impacto da pandemia do covid no enem 2020 em relação ao enem de 2019 a estudantes de menos de 25 anos que concluiram o ensino médio no ano da prova ou seguinte, chegamos a conclusão que:


*   A média de notas de 2020 foi maior que de 2019, porém como não contemplava o objetivo da análise não pudemos diferenciar estudantes de escolas publicas e privadas.
*   O total de inscritos por UF caiu em todos os estados.

*   A proporção de queda em relação a cor da pele ou etnia INDIGENA com 56.3%,
 PRETA com 54.2%, N/I (Não Informado) com 44.5%, PARDA com 49.0% e BRANCA com 32.8%.
*   Com a pandemia tivemos uma queda de alunos que fizeram a prova de 43,25%.


---





